In [1]:
import torch
import random
import datetime
import pandas as pd
import numpy as np
import os

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData, OpenCDP
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems
from src.embeddings import RecsysEmbedding

experiment_name = 'LogReg'
device = 'cuda:0'
seed = 7331
pkl_path = '../pkl/'


random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Модель

In [2]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, embedding, output_dim=1):
        super().__init__()
        self.embedding = embedding
        self.linear = torch.nn.Linear(2 * embedding.embedding_dim, output_dim)
    
    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)

        features = torch.cat(
            [
                item_embs,
                user_embs[:, :, None, :].repeat(1, 1, item_embs.size(-2), 1)
            ],
            dim = -1
        )
        return self.linear(features).squeeze(-1)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [3]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = LogisticRegression(
    RecsysEmbedding(
        d.n_items, 
        dummy_matrix, 
        embeddings='svd',
        embedding_dim=2
    ).to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device='cpu', lr=1e-2, num_epochs=5000, 
    silent=True
)

biulding affinity matrix...


3it [00:00, 3006.67it/s]


Test before learning: {'f1': 0.0, 'roc-auc': 0.0, 'accuracy': 0.75}


train... loss:0.7120252847671509:   0%|                                                                                                    | 2/5000 [00:00<17:45,  4.69it/s]

Val update: epoch: 0 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.0 | treshold: 0.01
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.0 | 


...

train... loss:0.6438579559326172:   0%|▍                                                                                                  | 23/5000 [00:04<16:04,  5.16it/s]

Val update: epoch: 21 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.4
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 


train... loss:0.6351967453956604:   0%|▍                                                                                                  | 25/5000 [00:04<16:32,  5.01it/s]

Val update: epoch: 25 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.47000000000000003
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(LogisticRegression(
   (embedding): RecsysEmbedding()
   (linear): Linear(in_features=4, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [4]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [5]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = LogisticRegression(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating LogReg with svd embeddings
Test before learning: {'f1': 0.15583325922489166, 'roc-auc': 0.5397244691848755, 'accuracy': 0.8541621565818787}


train... loss:34.17369136214256:   0%|▎                                                                                                 | 15/5000 [01:22<6:48:38,  4.92s/it]

Val update: epoch: 14 |accuracy: 0.4035112261772156 | f1: 0.18607716262340546 | auc: 0.5416983962059021 | treshold: 0.11
Test: accuracy: 0.38198941946029663 | f1: 0.18071456253528595 | auc: 0.531307578086853 | 


train... loss:33.66252315044403:   0%|▎                                                                                                 | 16/5000 [01:27<6:57:37,  5.03s/it]

Val update: epoch: 15 |accuracy: 0.4685017168521881 | f1: 0.1889752596616745 | auc: 0.5566744804382324 | treshold: 0.11
Test: accuracy: 0.4539448022842407 | f1: 0.1840384602546692 | auc: 0.5457164645195007 | 


...

train... loss:31.201329618692398:   0%|▍                                                                                                | 23/5000 [02:25<8:39:32,  6.26s/it]

Val update: epoch: 22 |accuracy: 0.7970371246337891 | f1: 0.2761264443397522 | auc: 0.6958919167518616 | treshold: 0.14
Test: accuracy: 0.7954937219619751 | f1: 0.2621288299560547 | auc: 0.6899195909500122 | 


train... loss:31.132576048374176:   0%|▍                                                                                                | 25/5000 [02:37<8:44:33,  6.33s/it]

Val update: epoch: 24 |accuracy: 0.7841075658798218 | f1: 0.2786654829978943 | auc: 0.6993419528007507 | treshold: 0.13
Test: accuracy: 0.7866261005401611 | f1: 0.26961609721183777 | auc: 0.6893941760063171 | 


train... loss:31.210504978895187:   1%|▌                                                                                                | 31/5000 [03:20<8:56:18,  6.48s/it]


In [6]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [7]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [8]:
for embeddings in ['explicit','neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = LogisticRegression(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings, 
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)
    
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items


Evaluating LogReg with explicit embeddings
Test before learning: {'f1': 0.6076813340187073, 'roc-auc': 0.5514298677444458, 'accuracy': 0.5413129925727844}


train... loss:66.78838258981705:   0%|                                                                                                  | 1/5000 [00:32<45:49:15, 33.00s/it]

Val update: epoch: 0 |accuracy: 0.6575968861579895 | f1: 0.7916035056114197 | auc: 0.7399385571479797 | treshold: 0.29000000000000004
Test: accuracy: 0.6538024544715881 | f1: 0.7886071801185608 | auc: 0.7424671053886414 | 


train... loss:60.841500490903854:   0%|                                                                                                 | 2/5000 [00:39<23:56:53, 17.25s/it]

Val update: epoch: 1 |accuracy: 0.7362985610961914 | f1: 0.818467915058136 | auc: 0.7860544919967651 | treshold: 0.45
Test: accuracy: 0.7344214916229248 | f1: 0.8163838386535645 | auc: 0.7836198806762695 | 


...

train... loss:52.140605598688126:   2%|██▎                                                                                             | 118/5000 [11:55<8:03:36,  5.94s/it]

Val update: epoch: 117 |accuracy: 0.7853721976280212 | f1: 0.8481467366218567 | auc: 0.8621437549591064 | treshold: 0.48000000000000004
Test: accuracy: 0.7836053967475891 | f1: 0.8461340069770813 | auc: 0.8630431294441223 | 


train... loss:52.150305688381195:   2%|██▎                                                                                             | 119/5000 [12:01<8:11:48,  6.05s/it]

Val update: epoch: 118 |accuracy: 0.7897982597351074 | f1: 0.846760094165802 | auc: 0.8623231649398804 | treshold: 0.52
Test: accuracy: 0.7899407744407654 | f1: 0.8462069630622864 | auc: 0.8633173108100891 | 


train... loss:52.31201529502869:   2%|██▎                                                                                              | 121/5000 [12:18<8:16:23,  6.10s/it]


# OpenCDP

In [ ]:
for group in ['cosmetics', 'multi']:
    for filename in os.listdir(pkl_path):    
        result = []
        if not filename.startswith(group):
            continue
        print(f"\n == {filename} ==")
        dataset = OpenCDP.load(os.path.join(pkl_path, filename))
        (
            train_loader, 
            val_loader,
            test_loader, 
            train_user_item_matrix, 
            train_num_items
        ) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=800)
    
        print(f"{len(dataset)} data points among {len(train_loader)} batches")
        for embeddings in ['neural', 'svd']:
            print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
        
            model = LogisticRegression(
                RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
                output_dim=1
            ).to(device)
        
            best_model, metrics = train(
                model, 
                train_loader, val_loader, test_loader, 
                device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
                silent=True
            )
            
            print(metrics)
            metrics['embeddings'] = embeddings
            result.append(metrics)
        pd.DataFrame(result).to_csv(f'results/{filename}_{experiment_name}.csv')
        del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items


 == cosmetics_10_8.pkl ==
115256 data points among 116 batches

Evaluating LogReg with neural embeddings
Test before learning: {'f1': 0.5768511891365051, 'roc-auc': 0.499575674533844, 'accuracy': 0.511569082736969}


train... loss:80.43690919876099:   0%|                                                                                              | 1/5000 [23:46<1980:23:18, 1426.17s/it]

Val update: epoch: 0 |accuracy: 0.5287438631057739 | f1: 0.6917363405227661 | auc: 0.5483413934707642 | treshold: 0.16
Test: accuracy: 0.5379822254180908 | f1: 0.6995948553085327 | auc: 0.545648455619812 | 


train... loss:77.57217019796371:   0%|                                                                                                | 2/5000 [24:01<827:54:45, 596.34s/it]

Val update: epoch: 1 |accuracy: 0.5628219842910767 | f1: 0.6952658295631409 | auc: 0.6105378866195679 | treshold: 0.38
Test: accuracy: 0.5707023739814758 | f1: 0.7025179862976074 | auc: 0.6083998680114746 | 


...

train... loss:80.08680200576782:   0%|▍                                                                                                | 22/5000 [04:45<18:10:48, 13.15s/it]

Val update: epoch: 21 |accuracy: 0.5271613001823425 | f1: 0.6900226473808289 | auc: 0.5587033033370972 | treshold: 0.37
Test: accuracy: 0.5363726019859314 | f1: 0.6978881359100342 | auc: 0.5544901490211487 | 


train... loss:79.93893045186996:   1%|▋                                                                                                | 34/5000 [07:26<18:07:06, 13.13s/it]


{'f1': 0.6978881359100342, 'roc-auc': 0.5544901490211487, 'accuracy': 0.5363726019859314}

 == cosmetics_5_2.pkl ==
121514 data points among 122 batches

Evaluating LogReg with neural embeddings
Test before learning: {'f1': 0.5238063335418701, 'roc-auc': 0.5068255662918091, 'accuracy': 0.5046188235282898}


train:   0%|                                                                                                                                       | 0/5000 [00:00<?, ?it/s]